In [ ]:
import itertools
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn import preprocessing
from IPython.display import display
from functions import *

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 161)
sns.set_theme()

: 

In [ ]:
data = pd.read_csv("./datasets/fr.openfoodfacts.org.products.csv", sep='\t', decimal='.')

## uniformisation du type des colones

In [ ]:
for name, col in data.iloc[:, [0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48]].iteritems():
    if len((col.dropna().apply(type).unique())) != 1:
        print(name)
del(name, col)

In [ ]:
data[['code', 'created_t', 'last_modified_t']].applymap(type).apply(
    lambda col: col.dropna().value_counts())

Hormis trois colones dont le code barre et les métadonnées, la seul raisons de l'hétérogénéité des colones est lié la présence de valeurs NaN identifiés comme réel. On choisit de convertir chaque colone vers son type principale

In [ ]:
data = data.convert_dtypes()

## Densité du dataset

le dataset est constitué d'un ensemble de plus de 300 000 individus constitué de 162 variables

In [ ]:
data.shape

on commence par évaluer la densité du dataset pour pour mieux évaluer la quantité l'information présente

In [ ]:
def densityByCountry (country=None):
    byCountry = data[data.countries.str.contains(country)] if country else data
    (100-byCountry.isna().sum().sort_values()/len(byCountry)*100).plot()
    plt.gca().set_xticks(range(0, byCountry.shape[1]+1, 20), range(0, byCountry.shape[1]+1, 20))
    plt.show()

densityByCountry()

la moitié des colones sont vides ou presque, un peu plus d'une trentaines sont remplis à 60% ou plus. on choisit d'examiner ces colones de plus prés pour mieux comprendre la nature de l'information disponible

In [ ]:
print(len(pd.cut(100-data.isna().sum().sort_values()/len(data)*100, range(60, 101, 10)).dropna()))
pd.cut(100-data.isna().sum().sort_values()/len(data)*100, range(60, 101, 10)).dropna()

les colones les plus remplies sont constitués de méta données :
- created_datetime, created_t : date de création de page sur open fact food
- last_modified_datetime, last_modified_t : date de dernière modification de page sur open fact food
- creator : contributeur de l'ajout initial à la base open fact food
- states, states_tags, states_fr : état de complétion de la saisie
- url : adresse web de la page du produit sur le site open fact food

=> ces données apportent peu d'information sur le produit en lui même et sont écartés pour l'instant

des données descriptive sur le produit :
- code : code barre EAN 13 du produit
- countries, countries_tags, countries_fr : pays ou à eu lieu la saisie, et donc la vente
- product_name, brands, brands_tags : nom du produit et marque

=> éviter la redondance et choisir une représentation unifié pour les pays et marques. le code barre est sortie de l'étude pour l'instant, au besoin vérifier la conformité et unicité

des données sur la qualité du le produit :
- ingredients_text : liste des ingrédient
- additives, additives_n : liste et nombres des additifs
- ingredients_from_palm_oil_n, ingredients_that_may_be_from_palm_oil_n: source d'huile de palme
- nutrition-score-uk_z, nutrition-score-fr_100g, nutrition_grade_fr : valeur nutritive

=> le nutriscore est la donnée la plus synthétique et porteuse d'information, il faudra vérifier si la redondance france / angleterre peux apporter de l'information. les autres informations sont écartés pour l'instant

des données quantitatives sur le produit :
- energy_100g, sugars_100g, saturated-fat_100g, sodium_100g : composante negative du nutriscore
- fiber_100g, proteins_100g : composante positive du du nutriscore
- salt_100g, fat_100g, carbohydrates_100g : données complémentaires possiblement corrélées 

=> ces données pourraient servir à inférer le nutriscore et seront conservées

note pour plus tard : serving_size
 

In [ ]:
food = data.dropna(axis=1, thresh=len(data)*0.6).copy()
food.drop(['code', 'created_datetime', 'created_t' , 'last_modified_datetime', 'last_modified_t', 'creator', 'states', 'states_tags', 'states_fr', 'url'], axis=1, inplace=True)
food.drop(['ingredients_text', 'additives', 'additives_n', 'ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n'], axis=1, inplace=True)
food.shape

## Choix de représentation unifié pour les pays

In [ ]:
print( data[['countries', 'countries_tags', 'countries_fr']].isnull().sum() )

Il y'a autant d'information manquante dans chaque colones représentant les pays

In [ ]:
for col in ['countries', 'countries_tags', 'countries_fr'] :
    print(f"{col} : {food[col].str.split(',').dropna().explode().nunique()}")
del(col)

pour une même quantité d'information, la colone countries_fr demandera le moins d'effort f'uniformisation

In [ ]:
food.drop(['countries', 'countries_tags'], axis=1, inplace=True)
food.shape

## Choix de représentation unifié pour les marques

In [ ]:
print( data[['brands', 'brands_tags']].isnull().sum() )

Il y'a une même quantitée d'information manquante dans chaque colones représentant les marques

In [ ]:
display(food['brands'].str.split(',').dropna().explode().nunique())
display(food['brands_tags'].str.split(',').dropna().explode().nunique())

à quantité d'information identique, la colone brands_tags demandera le moins d'effort f'uniformisation

In [ ]:
food.drop(['brands_tags'], axis=1, inplace=True)
food.shape

## Choix d'une représentation unifié pour le nutriscore

existe une part d'information dans l'une des données du nutriscore présente dans l'une où l'autre ne l'est pas

In [ ]:
(food['nutrition-score-uk_100g'].isna()^food['nutrition-score-fr_100g'].isna()).value_counts()

existe t'il un écart entre ces deux valeurs ?

In [ ]:
(food['nutrition-score-uk_100g'] - food['nutrition-score-fr_100g']).abs(
    ).describe(percentiles=[0.68, 0.96, 0.97])[['mean', 'std', '96%', '97%']]

il existe une différence pour un peu moins de 4% des nutriscore renseignés. un écart trop important peux être indicatif d'une erreur. elles sont gardés toutes les deux pour aider aux nettoyages des variables

## Choix d'une représentation unifié pour le sel et sodium

L'équivalent sel pour 100g est de 2,54 fois la quantitée de sodium pour 100g, ce qui se vérifie dans les données avec une erreur relative supérieur à 3% pour moins de 0.1% des produits présents

In [ ]:
saltSodium = food.loc[ (food.salt_100g>0) & (food.sodium_100g>0) ,['salt_100g', 'sodium_100g']]
display(((saltSodium.salt_100g - 2.54*saltSodium.sodium_100g) / saltSodium.salt_100g * 100).abs().describe(percentiles=[0.99, 0.999, 1])[['99%', '99.9%', 'max']])
del(saltSodium)

il existe un peu complémentarité dans les données qui pourront servir à remplir les valeurs manquantes après nettoyage

In [ ]:
(food.salt_100g.isna()^food.sodium_100g.isna()).value_counts()

# Nettoyage des variables choisies

## Nettoyage des nutriscores

Le nutriscore est issue du 'Nutrient Profiling Model' mis au point par la 'Food Standards Agency' (FSA) en 2004. il va de -15 (favorable) à 40 (défavorable)

In [ ]:
food[['nutrition-score-uk_100g', 'nutrition-score-fr_100g']].dtypes

les deux colones sont bien interprétés comme des entiers

In [ ]:
food[['nutrition-score-uk_100g', 'nutrition-score-fr_100g']].describe().loc[['min', 'max']]

l'ensemble des colones se référant au nutriscore sont conformes

Du nutriscore est dérivée le nutrigrade qui à pour fonction de rendre le score plus accessible au grand public. Il va de 'A' (favorable) à 'E' défavorable

In [ ]:
sorted(list(food['nutrition_grade_fr'].dropna().unique()))

la colone nutrigrade est conforme

l'écart de valeur entre nutrition-score-uk_100g et nutrition-score-fr_100g peux être indicatif d'une erreur dans le calcul des scores

In [ ]:
(food['nutrition-score-uk_100g'] - food['nutrition-score-fr_100g']).abs().value_counts().sort_index()[1:].plot.bar()
plt.show()

Les erreurs inférieures à 4 sont à la fois peux significative et peu nombreuses et sont gardés. Les autres sont marqués NA pour traitement ultérieur

In [ ]:
food.loc[(food['nutrition-score-uk_100g'] - food['nutrition-score-fr_100g']).abs()>=5,
    ['nutrition-score-uk_100g', 'nutrition-score-fr_100g']] = [pd.NA, pd.NA]

Une fois les écarts marqués comme erreurs, les colones nutrition-score-uk_100g et nutrition-score-fr_100g deviennent redondantes. On ne garde que nutrition-score-fr_100g

In [ ]:
food.drop(['nutrition-score-uk_100g'], axis=1, inplace=True)

## Nettoyage des valeurs nutritionnels

### nettoyages par analyse univariée

#### cas général

les valeurs nutritive pour 100g (hors énergie) sont des nombres réels compris entre 0 et 100

In [ ]:
(food.loc[:,'energy_100g':'salt_100g'].dtypes == 'Float64').all()

In [ ]:
temp = food.loc[:,'fat_100g':'salt_100g'].applymap(
    lambda x: (x<0) or (x>100), na_action='ignore').apply(pd.Series.value_counts)
print(f'{temp.loc[True,:].sum()} valeurs non conformes')
del(temp)

soit 246 valeurs non conformes que l'on remplace par NA pour autre traitement ultérieur

In [ ]:
for col in food.loc[:,'fat_100g':'salt_100g']:
    food.loc[(food[col]<0) | (food[col]>100), col] = pd.NA
food.loc[:,'fat_100g':'sodium_100g'].copy().describe().loc[['min', 'max']]

#### cas spécifique du Sodium

Concernant le sodium en relation avec le sel, à 100g de sel correspond 39.4g de sodium, il ne peut y avoir donc plus de 39.4g de sodium par 100g d'aliment

In [ ]:
temp = food.sodium_100g.dropna().apply(lambda x: (x<0) or (x>39.4)).value_counts()
print(f"{temp[True]} valeurs non conformes")
del(temp)

soit 156 valeurs non conformes que l'on remplace par NA pour autre traitement ultérieur

In [ ]:
food.loc[(food.sodium_100g<0) | (food.sodium_100g>39.4), 'sodium_100g'] = pd.NA
food.sodium_100g.describe().loc[['min', 'max']]

#### Cas spécifique de l'énergie

la quantité d'énergie max par gramme étant de 38kJ, l'énergie pour 100g doit être compris entre 0 et 3800Kj

In [ ]:
temp = food.energy_100g.dropna().apply(lambda x: (x<0) or (x>3800)).value_counts()
print(f"{temp[True]} valeurs non conformes")
del(temp)

soit 357 valeurs non conformes que l'on remplace par NA pour autre traitement ultérieur

In [ ]:
food.loc[(food.energy_100g<0) | (food.energy_100g>3800), 'energy_100g'] = pd.NA
food.energy_100g.describe().loc[['min', 'max']]

### nettoyages par analyse multivariée

In [ ]:
def zScoreAbs(col):
    return ((food[col]-food[col].mean())/food[col].std()).abs()

In [ ]:
def zMaxIs(col, cols):
    temp = pd.Series(True, index=food[col].index)
    for coln in cols :
        temp = temp & (food[col] > food[coln])
    return temp

In [ ]:
for col in ['carbohydrates_z', 'sugars_z', 'fiber_z']:
    food[col] = zScoreAbs(col.replace('z','100g'))

err3 = food.carbohydrates_100g < food.sugars_100g + food.fiber_100g
food.loc[err3 & zMaxIs('carbohydrates_z',['sugars_z', 'fiber_z']), 'carbohydrates_100g'] = pd.NA
food.loc[err3 & zMaxIs('sugars_z',['carbohydrates_z', 'fiber_z']), 'sugars_100g'] = pd.NA
food.loc[err3 & zMaxIs('fiber_z',['carbohydrates_z', 'sugars_z']), 'fiber_100g'] = pd.NA

err2 = food.carbohydrates_100g < food.fiber_100g
food.loc[err2 & zMaxIs('fiber_z',['carbohydrates_z']), 'fiber_100g'] = pd.NA
food.loc[err2 & zMaxIs('carbohydrates_z',['fiber_z']), 'carbohydrates_100g'] = pd.NA

err1 = food.carbohydrates_100g < food.sugars_100g
food.loc[err1 & zMaxIs('sugars_z',['carbohydrates_z']), 'sugars_100g'] = pd.NA
food.loc[err1 & zMaxIs('carbohydrates_z',['sugars_z']), 'carbohydrates_100g'] = pd.NA

print(err1.sum()+err2.sum()+err3.sum())
food = food.copy()

In [ ]:
for col in ['fat_z', 'saturated-fat_z']:
    food[col] = zScoreAbs(col.replace('z','100g'))

err = food.fat_100g < food['saturated-fat_100g']
food.loc[err & zMaxIs('saturated-fat_z',['fat_z']), 'saturated-fat_100g'] = pd.NA
food.loc[err & zMaxIs('fat_z',['saturated-fat_z']), 'fat_100g'] = pd.NA

print(err.sum())
food = food.copy()

In [ ]:
food['proteins_z'] = zScoreAbs('proteins_100g')

err4 = food[['fat_100g', 'carbohydrates_100g', 'proteins_100g']].sum(axis=1)>100
food.loc[err4 & zMaxIs('fat_z',['carbohydrates_z', 'proteins_z']), 'fat_100g'] = pd.NA
food.loc[err4 & zMaxIs('carbohydrates_z',['fat_z', 'proteins_z']), 'carbohydrates_100g'] = pd.NA
food.loc[err4 & zMaxIs('proteins_z',['fat_z', 'carbohydrates_z']), 'proteins_100g'] = pd.NA

err3 = food[['carbohydrates_100g', 'proteins_100g']].sum(axis=1)>100
food.loc[err3 & zMaxIs('carbohydrates_z',['proteins_z']), 'carbohydrates_100g'] = pd.NA
food.loc[err3 & zMaxIs('proteins_z',['carbohydrates_z']), 'proteins_100g'] = pd.NA

err2 = food[['fat_100g', 'proteins_100g']].sum(axis=1)>100
food.loc[err2 & zMaxIs('fat_z',['proteins_z']), 'fat_100g'] = pd.NA
food.loc[err2 & zMaxIs('proteins_z',['fat_z']), 'proteins_100g'] = pd.NA

err1 = food[['fat_100g', 'carbohydrates_100g']].sum(axis=1)>100
food.loc[err1 & zMaxIs('fat_z',['carbohydrates_z']), 'fat_100g'] = pd.NA
food.loc[err1 & zMaxIs('carbohydrates_z',['fat_z']), 'carbohydrates_100g'] = pd.NA

print(err1.sum()+err2.sum()+err3.sum()+err4.sum())
food = food.copy()

In [ ]:
food['energy_z'] = zScoreAbs('energy_100g')

err = ( (food['energy_100g'] - 17*food['proteins_100g'] - 37*food['fat_100g'] - 17*food['carbohydrates_100g'] - 8*food['fiber_100g']) / food['energy_100g'] ).abs() > 0.1
food.loc[err & zMaxIs('energy_z', ['proteins_z', 'fat_z', 'carbohydrates_z', 'fiber_z']), 'energy_100g'] = pd.NA
food.loc[err & zMaxIs('proteins_z', ['energy_z', 'fat_z', 'carbohydrates_z', 'fiber_z']), 'proteins_100g'] = pd.NA
food.loc[err & zMaxIs('fat_z', ['energy_z', 'proteins_z', 'carbohydrates_z', 'fiber_z']), 'fat_100g'] = pd.NA
food.loc[err & zMaxIs('carbohydrates_z', ['energy_z', 'proteins_z', 'fat_z', 'fiber_z']), 'carbohydrates_100g'] = pd.NA
food.loc[err & zMaxIs('fiber_z', ['energy_z', 'proteins_z', 'fat_z', 'carbohydrates_z']), 'fiber_100g'] = pd.NA

print(err.sum())
food = food.copy()

In [ ]:
del(col, err, err1, err2, err3, err4)

### référence des liquides

Les liquides au sens du nutriscore ont des valeurs nutritives faibles ou nulles qui pourraient les voir supprimé à tord, on cherche donc à les identifier

#### 1er source

In [ ]:
print(len(data[data.pnns_groups_2 == 'Alcoholic beverages'].index))
food.drop(data[data.pnns_groups_2 == 'Alcoholic beverages'].index, inplace=True)

The Nutri-Score modification applies to the following beverages : Mineral water, flavoured water,fruit juices, nectars, smoothies, vegetable juices, drinks with added sugar and/or sweeteners,teas, infusions or coffee reconstituted exclusively with water

In [ ]:
len(boissons1 := data[
   (data.pnns_groups_1.str.contains('Beverages', case=False) |
    data.pnns_groups_2.str.contains('beverages|juices', case=False)) &
   ~data.product_name.str.contains('plat|lait|milk|huile|oil|biscuit|cookie|gâteau|cake|lacté|soluble|soja|riz', case=False)
])

#### 2éme source

In [ ]:
boissons = 'boissons,|,boissons|boissons au|beverage|drink|water|juice|nectar|smoothie|lemonade|eau|jus|soda|limonades'
boissons = 'beverage|drink|water|juice|nectar|smoothie|tea|infusion|coffee|lemonade|boisson|eau|jus|thé|café|soda|limonades'
data[data.categories.str.contains(boissons, case=False) & 
    ~data.categories.str.contains('plat|lait|milk|huile|oil|biscuit|cookie|gâteau|cake|lacté', case=False) #s préparés
].shape[0]

soit 16021 boissons potentiels correspondant à la définition du nutriscore

In [ ]:
boissons = 'beverage|drink|water|juice|nectar|smoothie|tea|infusion|coffee|lemonade|boisson|eau|jus|thé|café|soda|limonades'
erreurs = 'plat|lait|milk|huile|oil|biscuit|cookie|gâteau|cake|lacté|soja'
len(boissons21 := data[data.categories.str.contains(boissons, case=False) &
    ~data.serving_size.str.contains('g') &
    ~data.categories.str.contains(erreurs, case=False) &
    ~data.product_name.str.contains(erreurs, case=False)
])

dont 2957 boissons vendus au litre et considérées comme fiables

In [ ]:
data[data.categories.str.contains(boissons, case=False) & 
    (data.serving_size.isna() | data.serving_size.str.contains('g')) &
    ~data.categories.str.contains('plat|lait|milk|huile|oil|biscuit|cookie|gâteau|cake|lacté', case=False) #s préparés
].shape[0]

et 13064 potentiel, parmi lesquels on écarte ceux vendus au grammes, 

In [ ]:
data[data.categories.str.contains(boissons, case=False) & 
    data.serving_size.str.contains('g') &
    ~data.categories.str.contains('plat|lait|milk|huile|oil|biscuit|cookie|gâteau|cake|lacté', case=False) #s préparés
].shape[0]

réduisant les potentiels à 9221, parmi lesquels on écarte les erreurs évidentes sur les noms de produits et catégories

In [ ]:
boissons = 'boissons,|,boissons|boissons au|beverage|drink|water|juice|nectar|smoothie|lemonade|eau|jus|soda|limonades'
erreurs = '|'.join([',plant-based foods', 'biscotte', 'bloc de', 'capsule', 'cerneaux', 'chiches', 'compote', 'dosette', 'dénoyauté', 'déshydraté', 'effilées', 'en conserve', 'en poudre', 'farine', 'farines', 'filets de', 'flour', 'foie', 'fond de', 'fruits à coque', 'jambon', 'maceta', 'morceaux', 'moulu', 'moulus', 'open beauty', 'plant-based foods,', 'pois', 'poissons', 'ravioles', 'ravioli', 'rillette', 'rillettes', 'rondelles', 'râpée', 'sachet', 'saucisse', 'saucisses', 'soluble', 'sucre', 'surgelés', 'tortillas', 'veau', 'viande', 'épeautre'])
len(boissons22 := data[
     data.categories.str.contains(boissons, case=False) & 
     data.serving_size.isna() &
    ~data.categories.str.contains('plat|lait|milk|huile|oil|biscuit|cookie|gâteau|cake|lacté', case=False) &
    ~data.categories.str.contains(erreurs, case=False) &
    ~data.product_name.str.contains(erreurs, case=False)
])

#### toutes sources confondus

soit 9538 boissons confirmées

In [ ]:
len(boissonsIndex := boissons1.index.union(
           boissons21.index).union(
           boissons22.index))

qui peuvent être étendue à 11264 produits du même nom

In [ ]:
len(boissonsIndex := data[data.product_name.isin(data.loc[boissonsIndex, 'product_name'])].index)

Sur un total de départ de 21534 potentiels dont la majorités sont soit des erreurs soit ne correspondent pas ne correspondent à la définition de boissons du nutriscore

In [ ]:
len(boissonsPotentielIndex := data[
     data.pnns_groups_1.str.contains('beverages', case=False) |
     data.pnns_groups_2.str.contains('beverages|juices', case=False) |
     data.categories.str.contains('boissons|beverage|drink|water|juice|nectar|smoothie|tea|infusion|coffee|lemonade|eau|jus|thé|café|soda|limonades', case=False)
].index)

pouvant être étendue à 47666 potentiels dont la majorités sont soit des erreurs soit ne correspondent pas ne correspondent à la définition de boissons du nutriscore

In [ ]:
len(boissonsPotentielIndex := data[data.product_name.isin(data.loc[boissonsPotentielIndex, 'product_name'])].index)

on choisie de marquer comme non liquide les éléments non potentiels parmi ceux étant référencés par le pnns et les catégories

In [ ]:
len(nourritureIndex := data[
    data.pnns_groups_1.notna() |
    data.pnns_groups_2.notna() |
    data.categories.notna()
].index.difference(boissonsPotentielIndex)) #TODO 

et on étends au produit de même nom

In [ ]:
len(nourritureIndex := data[data.product_name.isin(data.loc[nourritureIndex, 'product_name'])].index)

In [ ]:
del(boissons, boissons1, boissons21, boissons22, erreurs)

In [ ]:
food.loc[food.index.intersection(boissonsIndex), 'isBeverage'] = 1.0

In [ ]:
food.loc[food.index.intersection(nourritureIndex), 'isBeverage'] = 0.0

In [ ]:
len(food[food.isBeverage==1.0])

In [ ]:
len(food[food.isBeverage==0.0])

### nettoyage des valeurs non inférables

les produits sans nutriscore nécessite la présence de valeurs nutritives pour les évaluer. La qualité de cette évaluation dépends de la quantitée d'information présente pour chaque produit

In [ ]:
(food.loc[(food['nutrition_grade_fr'].isna() & food['nutrition-score-fr_100g'].isna()).index.difference(boissonsPotentielIndex), 
    'energy_100g':'sodium_100g'].isna().sum(axis=1).value_counts() / len(food) * 100).sort_index().plot.bar()
plt.show()

In [ ]:
# TODO : make a graph of it ?
((food.loc[(food['nutrition_grade_fr'].isna() & food['nutrition-score-fr_100g'].isna()).index.difference(boissonsPotentielIndex), 
    'energy_100g':'sodium_100g'].isna().sum(axis=1).value_counts() / len(food) * 100).sort_index()).loc[4:9].cumsum()

Soit 0.2% à 12.3% de d'information perdus selon le niveau de qualité d'information choisis

In [ ]:
food.loc[4,'energy_100g':'sodium_100g']

les produits ayant 8 valeurs absentes n'offrent pas de base informative suffisante pour inférer le nutriscore et représentent un peu moins de 11% des produits. Leurs nombres et la faible quantitée informative qu'ils apporteraient contribueraient à faire baisser la qualité de l'informations

les produits ayant 4, 5, 6 ou 7 valeurs absentes sont peu nombreux et la quantité d'information présente ne permettraient pas d'inférer une information de qualités. On choisit de ne pas les inclure

In [ ]:
indexDrop = food[(food['nutrition_grade_fr'].isna() & 
                  food['nutrition-score-fr_100g'].isna() & 
                 (food.loc[:,'energy_100g':'sodium_100g'].isna().sum(axis=1)>=4)
)].index.difference(boissonsPotentielIndex)
food.drop(indexDrop, inplace=True)
print(len(indexDrop)) # > 63000
del(indexDrop)

L'équivalence sel sodium peux être utilisée pour compléter 124 valeurs manquantes

## Nettoyage de la variable pays

### nombre de pays par produit

### Ensemble des pays présent dans le dataset

In [ ]:
print( f"nombre d'occurrence : {len(data.countries_fr.str.split(',').dropna().explode())}" )
print( f"cardinalité : {len(data.countries_fr.str.split(',').dropna().explode().value_counts())}")
print(f"{(n:=10)} pays les plus fréquents")
data.countries_fr.str.split(',').dropna().explode().value_counts().head(n).plot.bar()
plt.show()
del(n)

In [ ]:
temp = data.copy()

countriesClassificationFr = {
    'France':['Guadeloupe','Guyane','La Réunion','Martinique','Mayotte','Nouvelle-Calédonie','Polynésie française','Saint-Martin','Saint-Pierre-et-Miquelon','Wallis-et-Futuna'],
    'Royaume-Uni':['Angleterre','Écosse','pays de Galles','Irlande du Nord'],
    'Nouvelle-Zélande':['Îles Cook'],
}

def countryInclude(df, parent, children):
    df.loc[df.countries_fr.str.contains(children) & 
          ~df.countries_fr.str.contains(parent), 'countries_fr'
    ] = df.countries_fr.str.replace(children, f'{parent},{children}')

def countryIncludes(df, parent, subset):
    for children in subset:
        countryInclude(df, parent, children)

def countriesIncludes(df, countriesClassification):
    for parent, subset in countriesClassification.items():
        countryIncludes(df, parent, subset)

# countriesIncludes(temp, countriesClassificationFr)
# display(temp[data.countries_fr.str.contains('Guadeloupe')].countries_fr.head())
del(temp, countriesClassificationFr)


In [ ]:
temp = data.copy()

countriesCorrectionFr = {
    'Allemagne':['Deutschland','Duitsland',],
    'Arabie saoudite':['السعودية',],
    'Australie':['Australien',],
    'Azerbaïdjan':['Azərbaycan',],
    'Bahreïn':['البحرين',],
    'Belgique':['Belgie','Belgien',],
    'Canada':['Québec'],
    'Danemark':['Denemarken',],
    'Écosse':['Scotland'],
    'États-Unis':['Etats-unis',],
    'Espagne':['Spanje','Spanyolorszag',],
    'France':['Frankreich','Frankrijk','Puyricard','Franciaorszag',],
    'Hong Kong':['香港',],
    'Hongrie':['Magyarorszag',],
    'Inde':['भारत',],
    'Irak':['العراق','Other-العراق',],
    'Japon':['日本',],
    'Oman':['سلطنة-عمان',],
    'Pays-Bas':['Nederland',],
    'Pologne':['Szczecin',],
    'Portugal':['Portugalia',],
    'Québec':['Quebec',],
    'République tchèque':['Czech','Tschechien',],
    'Royaume-Uni':['Nagy-britannia','المملكة-المتحدة',],
    'Suisse':['Zwitserland',],
    'Suède':['Zweden',],
    'Tunisie':['تونس',],
    'Turquie':['Turkiye',],
}

def countryRemoveDuplicateFr(df):
    df.loc[:, 'countries_fr'] = df.countries_fr.apply(
        lambda countries: ",".join(dict.fromkeys(countries.split(','))) )

def countryCorrectionFr(df, country, mistake):
    df.loc[df.countries_fr.str.contains(mistake),
        'countries_fr'] = df.countries_fr.str.replace(mistake, country)

def countryCorrectionsFr(df, country, mistakes):
    for mistake in mistakes:
        countryCorrectionFr(df, country, mistake)

def countriesRemoveTagsFr(df):
    df.loc[df.countries_fr.str.contains(fr'^.{{2}}:|,.{{2}}:', regex=True), 
        'countries_fr'] = df.countries_fr.str.replace(fr'.{{2}}:', '', regex=True)

def countriesCorrectionsFr(df, countriesCorrection):
    countriesRemoveTagsFr(df)
    for country, mistakes in countriesCorrection.items():
        countryCorrectionsFr(df, country, mistakes)
    countryRemoveDuplicateFr(df.dropna())

# countriesCorrections(temp, countriesCorrectionFr)

del(temp)

In [ ]:
temp = pd.DataFrame(data={'countries_fr':['France,États-Unis,en:France']})
display(temp.countries_fr)
countriesRemoveTagsFr(temp)
display(temp.countries_fr)
countryRemoveDuplicateFr(temp)
display(temp.countries_fr)
del(temp)

In [ ]:
temp = data.copy()
print(len( temp.countries_fr.dropna().str.split(',').explode().value_counts().index.tolist() ))
countriesRemoveTagsFr(temp)
print(len( temp.countries_fr.dropna().str.split(',').explode().value_counts() ))
countriesCorrectionsFr(temp, countriesCorrectionFr)
print(len( temp.countries_fr.dropna().str.split(',').explode().value_counts() ))
del(temp, countriesCorrectionFr)


### Nettoyage de la variable marque

# Inférence des valeurs manquantes

## Analyse des valeurs manquantes

### taux de corrélation entre les valeurs nutritives

In [ ]:
food.loc[:,'energy_100g':'sodium_100g'].corr().style.background_gradient(cmap='coolwarm')

Hors sodium, tout les variables sont au moins légèrement corrélées à une voire plusieurs d'entre elle. détaillons celle entrant dans le calcul du nutriscore
- energy : fat(0.7), saturated-fat(0.6), carbohydrates (0.5), protein(0.3) fiber(0.3)
- sugar : carbohydrates(0.7), energy(0.3), protein(0.3)
- saturated-fat : fat(0.7), energy(0.6)
- proteins : energy(0.3), sugar(0.3), fat(0.2), fiber(0.2)
- fibres : energy(0.3), carbohydrates(0.3), proteins(0.2)
- sodium : aucune

### taux de remplissage des valeurs nutritives

In [ ]:
100-food.loc[:,'energy_100g':'sodium_100g'].isna().sum().sort_values()/len(data)*100

Hors fibre et graisse saturé, le taux de valeur manquante est relativement faible : aux alentour de 5% ou moins

### conclusion et choix des méthodes d'inférences

energy, proteins, sugars, carbohydrates, fat, saturated-fat : toute ces variables ont un taux de corrélation suffisamment important au regard du faible taux de valeur manquante pour essayer de les combler par iterativeimputer

le sodium : étant faiblement corrélé, l'inférer par iterativeimputer introduirait un bruit sans valeur informative. Une statistique descriptive telle que la médiane permettrait de combler sans influencer la série. Au vue du faible taux de valeur manquante, une valeur non aberrante serait aussi de faible influence sur la série

la fibre : le taux élevé de valeur manquantes au regard des autres variables incite à cherche s'il n'y pas de raison à cet écart

## inférence du sodium

### inférence par le sel

L'équivalence sel sodium ne peux être utilisée que pour compléter 2 valeurs manquantes

In [ ]:
print( (food.salt_100g.isna() & ~food.sodium_100g.isna()).sum() )
food.loc[food.salt_100g.isna() & ~food.sodium_100g.isna(), 'sodium_100g'] = food.salt_100g/2.54

L'information apportée par salt_100g est maintenant entièrement redondante et peux être abandonnée

In [ ]:
food.drop(['salt_100g'], axis=1, inplace=True)

### inférence métier / positionnel

le sodium est une composante négative du nutriscore et du nutrigrade, inférer par une valeur médiane ou moyenne stable pourrait contribuer à relever le score de produit qui aurait choisit de ne pas préciser dans l'intérêt de leur vente, on choisit de regarder la distribution en fonction du grade

In [ ]:
food[~food.nutrition_grade_fr.isna()].groupby('nutrition_grade_fr').sodium_100g.agg(['mean','median'])

In [ ]:
pd.cut(food[~food.nutrition_grade_fr.isna() & food.nutrition_grade_fr.isin([*'cde'])].sodium_100g, np.append(np.arange(0,1.5,0.3), 100)).value_counts().sort_index()

la formule du nutriscore attribut un nombre de point négatif maximum à partir de 900mg de sel, valeur qui reste assez présente dans la distribution. On choisit de l'attribuer au valeur manquante

In [ ]:
food.loc[food.sodium_100g.isna(), 'sodium_100g'] = 0.901

## Inférence des liquides

In [ ]:
# TODO : STOP !!!

## Inférence des valeurs nutritives

### inférence par iterativeimputer

#### fit_transform

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# columnsToImput = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g', 'isBeverage']
# imputed = pd.DataFrame(index=food.index, columns=columnsToImput, data=
#     IterativeImputer(random_state=0, tol=0.001 ,max_iter=100, missing_values=np.nan).fit_transform(
#         food[columnsToImput].replace({pd.NA: np.nan})))

# display(food[columnsToImput].describe().loc[['min','max']])
# display(imputed.describe().loc[['min','max']])

# temp1 = imputed.energy_100g.dropna().apply(
#     lambda x: (x<0) or (x>3800)).value_counts()
# temp2 = imputed.loc[:,'fat_100g':'proteins_100g'].applymap(
#     lambda x: (x<0) or (x>100), na_action='ignore').apply(pd.Series.value_counts)
# print(f"{temp1[True] + temp2.loc[True,:].sum()} valeurs non conformes pour {food[columnsToImput].isna().sum().sum()} valeurs imputées")
# del(temp1, temp2, columnsToImput, imputed)


#### fit and transform

In [ ]:
# rawToImput = food.isna().any(axis=1)
# columnsToImput = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g', 'isBeverage']
# imputed = pd.DataFrame(index=food[rawToImput].index, columns=columnsToImput, data=
#     IterativeImputer(random_state=0, tol=0.001 ,max_iter=100, missing_values=np.nan
#         ).fit(food.loc[~rawToImput, columnsToImput].replace({pd.NA: np.nan})
#         ).transform(food.loc[rawToImput,columnsToImput].replace({pd.NA: np.nan})) )
        
# display(food[columnsToImput].describe().loc[['min','max']])
# display(imputed.describe().loc[['min','max']])

# temp1 = imputed.energy_100g.dropna().apply(
#     lambda x: (x<0) or (x>3800)).value_counts()
# temp2 = imputed.loc[:,'fat_100g':'proteins_100g'].applymap(
#     lambda x: (x<0) or (x>100), na_action='ignore').apply(pd.Series.value_counts)
# print(f"{temp1[True] if True in temp1.index else 0 + temp2.loc[True,:].sum()} valeurs non conformes pour {food[columnsToImput].isna().sum().sum()} valeurs imputées")
# del(temp1, temp2, rawToImput, columnsToImput, imputed)

#### with fiber

In [ ]:
# rawToImput = food.isna().any(axis=1)
# columnsToImput = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'isBeverage']
# imputed = pd.DataFrame(index=food[rawToImput].index, columns=columnsToImput, data=
#     IterativeImputer(random_state=0, tol=0.001 ,max_iter=100, missing_values=np.nan
#         ).fit(food.loc[~rawToImput, columnsToImput].replace({pd.NA: np.nan})
#         ).transform(food.loc[rawToImput,columnsToImput].replace({pd.NA: np.nan})) )
        

# display(food[columnsToImput].describe().loc[['min','max']])
# display(imputed.describe().loc[['min','max']])

# temp1 = imputed.energy_100g.dropna().apply(
#     lambda x: (x<0) or (x>3800)).value_counts()
# temp2 = imputed.loc[:,'fat_100g':'proteins_100g'].applymap(
#     lambda x: (x<0) or (x>100), na_action='ignore').apply(pd.Series.value_counts)
# print(f"{temp1[True] if True in temp1.index else 0 + temp2.loc[True,:].sum()} valeurs non conformes pour {food[columnsToImput].isna().sum().sum()} valeurs imputées")
# del(temp1, temp2, rawToImput, columnsToImput, imputed)

#### with KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

rawToImput = food.isna().any(axis=1)
columnsToImput = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'isBeverage']
imputed = pd.DataFrame(index=food[rawToImput].index, columns=columnsToImput, data=
    IterativeImputer(random_state=0, estimator=KNeighborsRegressor(n_neighbors=3), tol=0.001 ,max_iter=100, missing_values=np.nan
        ).fit(food.loc[~rawToImput, columnsToImput].replace({pd.NA: np.nan})
        ).transform(food.loc[rawToImput,columnsToImput].replace({pd.NA: np.nan})) )
        

display(food[columnsToImput].describe().loc[['min','max']])
display(imputed.describe().loc[['min','max']])

#### résultat

In [ ]:
food.loc[rawToImput,columnsToImput] = imputed
food['isBeverage'] = food['isBeverage'].apply(round).astype(float)
display(food[columnsToImput].describe().loc[['min','max']])
print(food[columnsToImput].isna().sum().sum())
del(rawToImput, columnsToImput, imputed)

## Nettoyage du NutriScore et NutriGrade

### Nourriture Solide

In [ ]:
def toNutriGradeSolid(NutriScore):
    return (
        'a' if       NutriScore <= -1 else
        'b' if 0  <= NutriScore <=  2 else
        'c' if 3  <= NutriScore <=  10 else
        'd' if 11 <= NutriScore <=  18 else
        'e'
    )

In [ ]:
mask = (food.isBeverage == 0.0) & food.nutrition_grade_fr.notna() & food['nutrition-score-fr_100g'].notna()
error = (food.loc[mask, 'nutrition-score-fr_100g'].apply(toNutriGradeSolid) != food.loc[mask, 'nutrition_grade_fr'])
index = error[error].index
len(index)

In [ ]:
food.loc[index, ['nutrition-score-fr_100g', 'nutrition_grade_fr']] = (pd.NA, pd.NA)

In [ ]:
del(mask, error, index)

#### Nourriture Liquide

## inférence des scores

### inférence du nutriscore

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# rawToImputS = food['nutrition-score-fr_100g'].isna()
# rawToFitS = ~food['nutrition-score-fr_100g'].isna()
# columnsToFitS = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']
# columnToImputS = 'nutrition-score-fr_100g'

# models = {
#     'unscale' : food[columnsToFitS].copy(),
#     'standard' : pd.DataFrame(StandardScaler().fit_transform(food[columnsToFitS]), columns=columnsToFitS, index=food.index),
#     'robust' : pd.DataFrame(RobustScaler().fit_transform(food[columnsToFitS]), columns=columnsToFitS, index=food.index),
#     'MinMax' : pd.DataFrame(MinMaxScaler().fit_transform(food[columnsToFitS]), columns=columnsToFitS, index=food.index),
# }

# splits = {model : dict(zip(
#     ['xtrain', 'xtest', 'ytrain', 'ytest'], 
#     train_test_split(models[model].loc[rawToFitS], food.loc[rawToFitS, columnToImputS], random_state=0, train_size=0.8)
# )) for model in ['unscale', 'standard', 'robust', 'MinMax']}

# nMax = 10
# pd.DataFrame({
#     model : pd.Series(index=range(1,nMax+1), data=
#         [KNeighborsRegressor(n_neighbors=i)
#             .fit(splits[model]['xtrain'].astype(float), splits[model]['ytrain'].astype(int))
#             .score(splits[model]['xtest'].astype(float), splits[model]['ytest'].astype(int))
#         for i in range(1,nMax+1)]) 
#     for model in ['unscale', 'standard', 'robust', 'MinMax']}).plot()
# del(rawToImputS, rawToFitS, columnsToFitS, columnToImputS, models, splits, nMax)

In [ ]:
rawToImputS = food['nutrition-score-fr_100g'].isna()
rawToFitS = ~food['nutrition-score-fr_100g'].isna()
columnsToFitS = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g', 'isBeverage']
columnToImputS = 'nutrition-score-fr_100g'

models = {
    'unscale' : food[columnsToFitS].copy(),
    'standard' : pd.DataFrame(StandardScaler().fit_transform(food[columnsToFitS]), columns=columnsToFitS, index=food.index),
    'robust' : pd.DataFrame(RobustScaler().fit_transform(food[columnsToFitS]), columns=columnsToFitS, index=food.index),
    'MinMax' : pd.DataFrame(MinMaxScaler().fit_transform(food[columnsToFitS]), columns=columnsToFitS, index=food.index),
}

del(columnsToFitS)
splits = {model : dict(zip(
    ['xtrain', 'xtest', 'ytrain', 'ytest'], 
    train_test_split(models[model].loc[rawToFitS], food.loc[rawToFitS, columnToImputS], random_state=0, train_size=0.8)
)) for model in ['unscale', 'standard', 'robust', 'MinMax']}
nMax = 10
pd.DataFrame({
    model : pd.Series(index=range(1,nMax+1), data=
        [KNeighborsRegressor(n_neighbors=i)
            .fit(splits[model]['xtrain'].astype(float), splits[model]['ytrain'].astype(int))
            .score(splits[model]['xtest'].astype(float), splits[model]['ytest'].astype(int))
        for i in range(1,nMax+1)]) 
    for model in ['unscale', 'standard', 'robust', 'MinMax']}).plot()

In [ ]:
nMax = 15
pd.DataFrame({
    'robust' : pd.Series(index=range(1,nMax+1), data=
        [KNeighborsRegressor(n_neighbors=i)
            .fit(splits['robust']['xtrain'].astype(float), splits['robust']['ytrain'].astype(int))
            .score(splits['robust']['xtest'].astype(float), splits['robust']['ytest'].astype(int))
        for i in range(1,nMax+1)]) 
    }).plot()

In [ ]:
nMax = 100
scores = pd.Series([
    (lambda xtrain, xtest, ytrain, ytest:
        KNeighborsRegressor(n_neighbors=5).fit(xtrain, ytrain).score(xtest, ytest))(
            *train_test_split(models['robust'].loc[rawToFitS].astype(float), food.loc[rawToFitS, columnToImputS].astype(int), random_state=rand, train_size=0.8)
)for rand in range(nMax)])

scores.plot.kde()
display(scores.describe()[['mean','std']])
del(nMax, scores, splits)

In [ ]:
imputed = pd.Series(index=food[rawToImputS].index, dtype=float, data=
    KNeighborsRegressor(n_neighbors=5)
        .fit(X=models['robust'].loc[rawToFitS], y=food.loc[rawToFitS, columnToImputS])
        .predict(X=models['robust'].loc[rawToImputS]) )
imputed.describe().loc[['min', 'max']]
del(rawToFitS, models)

In [ ]:
food.loc[rawToImputS, columnToImputS] = imputed.apply(round)
display(food['nutrition-score-fr_100g'].isna().sum())
del(rawToImputS, columnToImputS, imputed)

Inférence 

### inférence du nutrigrade

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [ ]:
# rawToImputG = food[food['nutrition_grade_fr'].isna()].index
# rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
# columnsToFitG = ['nutrition-score-fr_100g', 'isBeverage']
# columnToImputG = 'nutrition_grade_fr'

# models = {
#     'unscale' : food[columnsToFitG].copy(),
#     'standard' : pd.DataFrame(StandardScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
#     'robust' : pd.DataFrame(RobustScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
#     'MinMax' : pd.DataFrame(MinMaxScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
# }

# splits = {model : dict(zip(
#     ['xtrain', 'xtest', 'ytrain', 'ytest'], 
#     train_test_split(models[model].loc[rawToFitG], food.loc[rawToFitG, columnToImputG], random_state=0, train_size=0.8)
# )) for model in ['unscale', 'standard', 'robust', 'MinMax']}

# nMax = 10
# pd.DataFrame({
#     model : pd.Series(index=range(1,nMax+1), data=
#         [KNeighborsClassifier(n_neighbors=i)
#             .fit(splits[model]['xtrain'].astype(float), splits[model]['ytrain'])
#             .score(splits[model]['xtest'].astype(float), splits[model]['ytest'])
#         for i in range(1,nMax+1)]) 
#     for model in ['unscale', 'standard', 'robust', 'MinMax']}).plot()
# del(rawToImputG, rawToFitG, columnsToFitG, columnToImputG, models, splits, nMax)

In [ ]:
# rawToImputG = food[(food.isBeverage==0) & (food['nutrition_grade_fr'].isna())].index
# rawToFitG = food[(food.isBeverage==0) & ~food['nutrition_grade_fr'].isna()].index
# columnsToFitG = ['nutrition-score-fr_100g']
# columnToImputG = 'nutrition_grade_fr'

# models = {
#     'unscale' : food[columnsToFitG].copy(),
#     'standard' : pd.DataFrame(StandardScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
#     'robust' : pd.DataFrame(RobustScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
#     'MinMax' : pd.DataFrame(MinMaxScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
# }


# splits = {model : dict(zip(
#     ['xtrain', 'xtest', 'ytrain', 'ytest'], 
#     train_test_split(models[model].loc[rawToFitG], food.loc[rawToFitG, columnToImputG], random_state=0, train_size=0.8)
# )) for model in ['unscale', 'standard', 'robust', 'MinMax']}

# nMax = 10
# pd.DataFrame({
#     model : pd.Series(index=range(1,nMax+1), data=
#         [KNeighborsClassifier(n_neighbors=i)
#             .fit(splits[model]['xtrain'].astype(float), splits[model]['ytrain'])
#             .score(splits[model]['xtest'].astype(float), splits[model]['ytest'])
#         for i in range(1,nMax+1)]) 
#     for model in ['unscale', 'standard', 'robust', 'MinMax']}).plot()
# del(rawToImputG, rawToFitG, columnsToFitG, columnToImputG, models, splits, nMax)

In [ ]:
rawToImputG = food[(food.isBeverage==1) & (food['nutrition_grade_fr'].isna())].index
rawToFitG = food[(food.isBeverage==1) & (~food['nutrition_grade_fr'].isna())].index
columnCandidates = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g', 'isBeverage']
columnsToFitG = ['nutrition-score-fr_100g'] + columnCandidates
columnToImputG = 'nutrition_grade_fr'

models = {
    'unscale' : food[columnsToFitG].copy(),
    'standard' : pd.DataFrame(StandardScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
    'robust' : pd.DataFrame(RobustScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
    'MinMax' : pd.DataFrame(MinMaxScaler().fit_transform(food[columnsToFitG]), columns=columnsToFitG, index=food.index),
}

splits = {model : dict(zip(
    ['xtrain', 'xtest', 'ytrain', 'ytest'], 
    train_test_split(models[model].loc[rawToFitG], food.loc[rawToFitG, columnToImputG], random_state=3, train_size=0.95)
)) for model in ['unscale', 'standard', 'robust', 'MinMax']}
nMax = 10
pd.DataFrame({
    model : pd.Series(index=range(1,nMax+1), data=
        [KNeighborsClassifier(n_neighbors=i)
            .fit(splits[model]['xtrain'].astype(float), splits[model]['ytrain'])
            .score(splits[model]['xtest'].astype(float), splits[model]['ytest'])
        for i in range(1,nMax+1)]) 
    for model in ['unscale', 'standard', 'robust', 'MinMax']}).plot()
#del(rawToImputG, rawToFitG, columnsToFitG, columnToImputG, models, splits, nMax)

In [ ]:
# rawToImputG = food[food['nutrition_grade_fr'].isna()].index
# rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
# columnCandidates = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g', 'isBeverage']
# columnReferences = ['nutrition-score-fr_100g']
# columnToImputG = 'nutrition_grade_fr'

# robust = pd.DataFrame(RobustScaler().fit_transform(food[columnReferences+columnCandidates]), columns=columnReferences+columnCandidates, index=food.index)

# rawtrain, rawtest, ytrain, ytest = train_test_split(rawToFitG, food.loc[rawToFitG, columnToImputG], train_size=0.8, random_state=0)

# def KNeighborsClassifierScorer(n, columnsToFit):
#     return (KNeighborsClassifier(n_neighbors=n)
#         .fit(robust.loc[rawtrain, columnsToFit], ytrain)
#         .score(robust.loc[rawtest, columnsToFit], ytest))

# nMin, nMax= 4, 15
# Results = pd.DataFrame({
#             candidate : pd.Series(index=range(nMin, nMax+1), data=[
#                 KNeighborsClassifierScorer(n, columnReferences+[candidate])
#                 for n in range(nMin,nMax+1)])
#             for candidate in columnCandidates})
# Results['reference'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, columnReferences) for n in range(nMin,nMax+1)])

# fig = plt.figure(dpi=200)
# Results.plot(ax = plt.gca())

In [ ]:
# rawToImputG = food[food['nutrition_grade_fr'].isna()].index
# rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
# columnCandidates = ['fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g', 'isBeverage']
# columnReferences = ['nutrition-score-fr_100g', 'energy_100g']
# columnToImputG = 'nutrition_grade_fr'

# robust = pd.DataFrame(RobustScaler().fit_transform(food[columnReferences+columnCandidates]), columns=columnReferences+columnCandidates, index=food.index)

# rawtrain, rawtest, ytrain, ytest = train_test_split(rawToFitG, food.loc[rawToFitG, columnToImputG], train_size=0.8, random_state=0)

# def KNeighborsClassifierScorer(n, columnsToFit):
#     return (KNeighborsClassifier(n_neighbors=n)
#         .fit(robust.loc[rawtrain, columnsToFit], ytrain)
#         .score(robust.loc[rawtest, columnsToFit], ytest))

# nMin, nMax= 4, 15
# Results = pd.DataFrame({
#             candidate : pd.Series(index=range(nMin, nMax+1), data=[
#                 KNeighborsClassifierScorer(n, columnReferences+[candidate])
#                 for n in range(nMin,nMax+1)])
#             for candidate in columnCandidates})
# Results['reference'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, columnReferences) for n in range(nMin,nMax+1)])

# fig = plt.figure(dpi=200)
# Results.plot(ax = plt.gca())

In [ ]:
# rawToImputG = food[food['nutrition_grade_fr'].isna()].index
# rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
# columnCandidates = ['fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g', 'isBeverage']
# columnReferences = ['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g']
# columnToImputG = 'nutrition_grade_fr'

# robust = pd.DataFrame(RobustScaler().fit_transform(food[columnReferences+columnCandidates]), columns=columnReferences+columnCandidates, index=food.index)

# rawtrain, rawtest, ytrain, ytest = train_test_split(rawToFitG, food.loc[rawToFitG, columnToImputG], train_size=0.8, random_state=0)

# def KNeighborsClassifierScorer(n, columnsToFit):
#     return (KNeighborsClassifier(n_neighbors=n)
#         .fit(robust.loc[rawtrain, columnsToFit], ytrain)
#         .score(robust.loc[rawtest, columnsToFit], ytest))

# nMin, nMax= 4, 15
# Results = pd.DataFrame({
#             candidate : pd.Series(index=range(nMin, nMax+1), data=[
#                 KNeighborsClassifierScorer(n, columnReferences+[candidate])
#                 for n in range(nMin,nMax+1)])
#             for candidate in columnCandidates})
# Results['reference'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, columnReferences) for n in range(nMin,nMax+1)])

# fig = plt.figure(dpi=200)
# Results.plot(ax = plt.gca())

In [ ]:
# rawToImputG = food[food['nutrition_grade_fr'].isna()].index
# rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
# columnCandidates = ['fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']
# columnReferences = ['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'isBeverage']
# columnToImputG = 'nutrition_grade_fr'

# robust = pd.DataFrame(RobustScaler().fit_transform(food[columnReferences+columnCandidates]), columns=columnReferences+columnCandidates, index=food.index)

# rawtrain, rawtest, ytrain, ytest = train_test_split(rawToFitG, food.loc[rawToFitG, columnToImputG], train_size=0.8, random_state=0)

# def KNeighborsClassifierScorer(n, columnsToFit):
#     return (KNeighborsClassifier(n_neighbors=n)
#         .fit(robust.loc[rawtrain, columnsToFit], ytrain)
#         .score(robust.loc[rawtest, columnsToFit], ytest))

# nMin, nMax= 4, 15
# Results = pd.DataFrame({
#             candidate : pd.Series(index=range(nMin, nMax+1), data=[
#                 KNeighborsClassifierScorer(n, columnReferences+[candidate])
#                 for n in range(nMin,nMax+1)])
#             for candidate in columnCandidates})
# Results['reference'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, columnReferences) for n in range(nMin,nMax+1)])

# fig = plt.figure(dpi=200)
# Results.plot(ax = plt.gca())

In [ ]:
rawToImputG = food[food['nutrition_grade_fr'].isna()].index
rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
columnCandidates = ['fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']
columnReferences = ['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'isBeverage']
columnToImputG = 'nutrition_grade_fr'

rawtrain, rawtest, ytrain, ytest = train_test_split(rawToFitG, food.loc[rawToFitG, columnToImputG], train_size=0.8, random_state=0)

robust = pd.DataFrame(RobustScaler()
    .fit(food.loc[rawtrain, columnReferences+columnCandidates])
    .transform(food.loc[:, columnReferences+columnCandidates])
, columns=columnReferences+columnCandidates, index=food.index)

def KNeighborsClassifierScorer(n, columnsToFit):
    return (KNeighborsClassifier(n_neighbors=n)
        .fit(robust.loc[rawtrain, columnsToFit], ytrain)
        .score(robust.loc[rawtest, columnsToFit], ytest))

nMin, nMax= 4, 15
Results = pd.DataFrame()
Results['nutri'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, ['nutrition-score-fr_100g']) for n in range(nMin,nMax+1)])
Results['nutri+enr'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, ['nutrition-score-fr_100g', 'energy_100g']) for n in range(nMin,nMax+1)])
Results['nutri+enr+car'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, ['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g']) for n in range(nMin,nMax+1)])
Results['nutri+enr+car+bev'] = pd.Series(index=range(nMin, nMax+1), data=[KNeighborsClassifierScorer(n, ['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'isBeverage']) for n in range(nMin,nMax+1)])

fig = plt.figure(dpi=200)
Results.plot(ax = plt.gca())

In [ ]:
# test de l'hypothése de perte de ligne dans les ensembles
print("l'ensemble des lignes dont il faut apprendre plus celle à imputer est constitutif du dataset filtré : ",
    (len(rawToImputG)+len(rawToFitG) == len(food)) &
     food.index.equals(rawToImputG.union(rawToFitG)) )
print("l'ensemble des lignes dont il faut apprendre plus celle a tester est constitutif de l'ensemble des nutrigrade non nul : ",
    (len(rawtrain)+len(rawtest) == food['nutrition_grade_fr'].notna().sum()) &
     rawToFitG.equals(rawtrain.union(rawtest)) )
print("l'ensemble des ligne mis à l'échelle est constitutif du dataset filtré : ",
    (len(robust) == len(food)) &
     robust.index.equals(food.index) )

In [ ]:
# test de l'hypothése d'un problème d'index
rawToImputG = food[food['nutrition_grade_fr'].isna()].index
rawToFitG = food[~food['nutrition_grade_fr'].isna()].index
columnCandidates = ['fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']
columnReferences = ['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'isBeverage']
columnToImputG = 'nutrition_grade_fr'

rawtrain, rawtest, ytrain, ytest = train_test_split(rawToFitG, food.loc[rawToFitG, columnToImputG], train_size=0.8, random_state=0)

robust = pd.DataFrame((scaler := RobustScaler())
    .fit(food.loc[rawtrain, columnReferences+columnCandidates])
    .transform(food.loc[:, columnReferences+columnCandidates])
, columns=columnReferences+columnCandidates, index=food.index)

reverse = pd.DataFrame(scaler.inverse_transform(robust)
, columns=columnReferences+columnCandidates, index=food.index)

display("test", data[['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']].loc[12:15])
display(food[['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']].loc[12:15])
display(reverse[['nutrition-score-fr_100g', 'energy_100g', 'carbohydrates_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']].loc[12:15])

del(reverse)

In [ ]:
STOP HERE !!!

In [ ]:
del(rawToImputG, rawToFitG, columnToImputG, robust, rawtrain, rawtest, ytrain, ytest, nMax, Results)

In [ ]:
rawToImputG = food['nutrition_grade_fr'].isna()
rawToFitG = ~food['nutrition_grade_fr'].isna()
columnsToFitG = ['nutrition-score-fr_100g']
columnToImputG = 'nutrition_grade_fr'

imputed = pd.Series(index=food[rawToImputG].index, dtype=pd.StringDtype(), data=
    KNeighborsClassifier(n_neighbors=12
        ).fit(X=food.loc[rawToFitG, columnsToFitG].replace({pd.NA: np.nan}), y=food.loc[rawToFitG, columnToImputG].replace({pd.NA: np.nan})
        ).predict(X=food.loc[rawToImputG, columnsToFitG].replace({pd.NA: np.nan})) )
imputed.value_counts().sort_index()

In [ ]:
food.loc[rawToImputG, columnToImputG] = imputed
food.nutrition_grade_fr.isna().any()

# Analyse exploratoire

## Univarié et bivarié

In [ ]:
cols = ['nutrition_grade_fr', 'nutrition-score-fr_100g', 'energy_100g', 'sugars_100g', 'saturated-fat_100g', 'sodium_100g', 'fiber_100g', 'proteins_100g']
sns.pairplot(food.groupby('nutrition_grade_fr').sample(100)[cols], hue='nutrition_grade_fr')


## Multivarié

### Acp

In [ ]:
food_pca = food.loc[~food.product_name.isna(),'energy_100g':'nutrition-score-fr_100g']

# Centrage et Réduction
X = preprocessing.StandardScaler().fit_transform(food_pca)

# Calcul des composantes principales
pca = decomposition.PCA(n_components=len(food_pca.columns)).fit(X)

# Eboulis des valeurs propres
display_scree_plot(pca)

In [ ]:
#load_ext autoreload
#autoreload 2
from functions import *

# Cercle des corrélations
pcs = pca.components_
display_circles(pca.components_, len(food_pca.columns), pca, [(0,1),(2,3),(4,5)], labels = food_pca.columns)

In [ ]:
# Projection des individus
food_grade = food[food['nutrition_grade_fr'].isin(list('ace'))]
food_pca = food_grade.groupby('nutrition_grade_fr').sample(100).loc[:,'energy_100g':'nutrition-score-fr_100g']

# Centrage et Réduction
X = preprocessing.StandardScaler().fit_transform(food_pca)

# Calcul des composantes principales
pca = decomposition.PCA(n_components=len(food_pca.columns)).fit(X)

display_factorial_planes(pca.transform(X), len(food_pca.columns), pca, [(0,1),(2,3),(4,5)], alpha=1,
    illustrative_var = np.array(food.loc[food_pca.index, 'nutrition_grade_fr'].replace({pd.NA: np.nan})))

### Anova - mange t'on aussi bien dans les 6 pays les plus représentés ?

In [ ]:
topCountries = data.countries_fr.str.split(',').explode().value_counts().head(6).index.tolist()
topCountries

In [ ]:
anova_df = pd.DataFrame({
    'country' : [country for country in topCountries for i in range(1000) ],
    'score' : pd.concat([food.loc[food.countries_fr.str.contains(country), 'nutrition-score-fr_100g'].sample(1000).astype(float) for country in topCountries]).reset_index(drop=True) })
print(anova_df.score.mean())
anova_df.groupby('country').mean().sort_values('score').transpose()

on se pose ici la question de savoir si le nutriscore Connaît une variation significative selon les pays. Pour les pays étudiés la moyenne globale est de **8.3**, allant de **6.39** pour l'espagne à **9.23** pour l'allemagne, le score le plus bas étant un meilleur score.

In [ ]:
sns.boxplot(x='country', y='score', data=anova_df, showmeans=True)

On se pose la question de la significativité de ces écarts avec une possibilité qu'ils soient du au hasard. au vue la taille des échantillons, on se fixe un seuil de 0.1%

H0 : on mange aussi bien en moyenne dans chaqu'un des 5 pays les plus représentés  
<=> U(us) = U(fr) = U(ch) = U(de) = U(en) = U(es)

H1 : on ne mange pas aussi bien en moyenne dans chaqu'un des 5 pays les plus représentés  
<=> Toute les moyennes ne sont pas égales

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('score ~ C(country)', data=anova_df).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
aov_table

on admet donc qu'il existe une différence statistiquement significative entre les moyennes du nutriscore selon les pays. On se pose la question de savoir si cette différence apporte une information pertinente

In [ ]:
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    return aov[['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']]

anova_table(aov_table)

avec un eta carré inférieur à 0.01, on conclue que si les différences sont significative, elle ne sont pas porteuse de sens et on rejette la corrélation entre nutriscore et pays

# --- Autre ---

In [ ]:
def scoreByCountry(country=None):
    score = data[['nutrition_grade_fr', 'nutrition-score-fr_100g']]
    if country:
        score = data[data.countries.str.contains(country)][['nutrition_grade_fr', 'nutrition-score-fr_100g']]
    else :
        score = data[['nutrition_grade_fr', 'nutrition-score-fr_100g']]
    score = score.assign( grade_or_score=score['nutrition_grade_fr'].combine_first(score['nutrition-score-fr_100g']))
    pd.concat(axis=1, objs=[
        (100-score.isnull().sum()/len(score) * 100)[::-1],
        (score.isnull().sum()/len(score) * 100)[::-1]]
    ).rename(columns={0:'not null',1:'null'}
    ).plot(kind='barh', stacked=True, color=['#306645','#4c4c4c'], figsize=(10,2) #green, gray
    ).legend(loc='upper left')
    plt.show()

scoreByCountry()
scoreByCountry(country='France')
scoreByCountry(country='États-Unis')

In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'energy_100g', 'sugars_100g', 'saturated-fat_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g'
]].copy()

mostFilled = (1-temp.isnull().sum().sort_values() / len(temp)).index.tolist()
temp = temp[mostFilled]

sns.heatmap(temp.isnull().sort_values(mostFilled, ascending=False).reset_index().drop('index', axis=1))
del(temp)

In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'energy_100g', 'sugars_100g', 'saturated-fat_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g'
]].copy()

temp = data.loc[temp[temp.isnull().all(axis=1)].index]
print(len(temp))
display(pd.cut(100-temp.isna().sum().sort_values()/len(temp)*100, range(10, 101, 10)).dropna())

del(temp)

In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'energy_100g', 'sugars_100g', 'saturated-fat_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g'
]].copy()

temp = data.loc[temp[temp.isnull().all(axis=1)].index][[
    'pnns_groups_2', 'pnns_groups_1', 'categories_fr', 'main_category_fr', 'ingredients_text', 'labels_fr'
]].copy()

mostFilled = (1-temp.isnull().sum().sort_values() / len(temp)).index.tolist()
sns.heatmap(temp[mostFilled].isnull().sort_values(mostFilled, ascending=False).reset_index().drop('index', axis=1))

del(temp)

In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'energy_100g', 'sugars_100g', 'saturated-fat_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g'
]].copy()

temp = data.loc[temp[temp.isnull().all(axis=1)].index][[
    'pnns_groups_2', 'pnns_groups_1', 'categories_fr', 'main_category_fr', 'ingredients_text', 'labels_fr'
]].copy()

# print( data.countries_fr.str.split(',').dropna().apply(len).value_counts().sort_index(kind = 'mergesort') )
# temp.categories_fr.str.split(',').dropna().value_counts()

# 183 : display(len(data[data.categories_fr.str.contains('Non alimentaire')]))
print(len(temp))
print(temp.pnns_groups_2.dropna().shape)
display(temp.pnns_groups_2.value_counts())

del(temp)

In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'sugars_100g', 'carbohydrates_100g'
]].copy()

display( data[['sugars_100g', 'carbohydrates_100g']].corr().style.background_gradient(cmap='coolwarm') )
display( temp[['sugars_100g', 'carbohydrates_100g']].corr().style.background_gradient(cmap='coolwarm') )
sns.heatmap(temp.isnull())
del(temp)

In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'saturated-fat_100g', 'fat_100g', 'cholesterol_100g', 'trans-fat_100g'
]].copy()

display( data[['saturated-fat_100g', 'fat_100g', 'cholesterol_100g', 'trans-fat_100g']].corr().style.background_gradient(cmap='coolwarm') )
display( temp[['saturated-fat_100g', 'fat_100g', 'cholesterol_100g', 'trans-fat_100g']].corr().style.background_gradient(cmap='coolwarm') )
sns.heatmap(temp.isnull())
del(temp)


In [ ]:
temp = data[data['nutrition-score-fr_100g'].isnull()][[
    'fiber_100g','carbohydrates_100g', 'sugars_100g', 'vitamin-a_100g', 'iron_100g'
]].copy()

display( data[['fiber_100g','carbohydrates_100g', 'sugars_100g', 'vitamin-a_100g', 'iron_100g']].corr().style.background_gradient(cmap='coolwarm') )
display( temp[['fiber_100g','carbohydrates_100g', 'sugars_100g', 'vitamin-a_100g', 'iron_100g']].corr().style.background_gradient(cmap='coolwarm') )
sns.heatmap(temp.isnull())
del(temp)

In [ ]:
(data.isna().sum()/len(data)*100).plot.hist(cumulative=True, bins=range(0, 101, 10))

In [ ]:
categorie = data[['categories', 'categories_tags', 'categories_fr','pnns_groups_1', 'pnns_groups_2', 'labels', 'labels_tags', 'labels_fr']]
pd.concat(axis=1, objs=[
    (100-categorie.isnull().sum()/len(categorie) * 100)[::-1],
    (categorie.isnull().sum()/len(categorie) * 100)[::-1]]
).rename(columns={0:'not null',1:'null'}
).plot(kind='barh', stacked=True, color=['#306645','#4c4c4c'], figsize=(10,6) #green, gray
).legend(loc='upper left')
plt.show()

In [ ]:
unscore = data[data.nutrition_grade_fr.isnull()].categories_tags
print (f"defined : {len(unscore)/len(data)*100}, numbers of bin : {len(score.value_counts())}")
display(unscore)

In [ ]:
calcul = data[['energy_100g', 'sugars_100g', 'saturated-fat_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g']]
pd.concat(axis=1, objs=[
    (100-calcul.isnull().sum()/len(calcul) * 100)[::-1],
    (calcul.isnull().sum()/len(calcul) * 100)[::-1]]
).rename(columns={0:'not null',1:'null'}
).plot(kind='barh', stacked=True, color=['#306645','#4c4c4c'], figsize=(10,6) #green, gray
).legend(loc='upper left')
plt.show()

In [ ]:
data.loc[:,'fat_100g':].dropna(axis=1, how='all').corr().fat_100g.sort_values(ascending=False).head(1)

In [ ]:
data['capric-acid_100g'].isnull().sum()/len(data) * 100

In [ ]:
data.loc[:,'fat_100g':].dropna(axis=1, how='all').corr()['saturated-fat_100g'].sort_values(ascending=False).head(5)

In [ ]:
data['chloride_100g'].isnull().sum()/len(data) * 100

## nettoyage

les valeurs étant donnés pour 100g, elles doivent être comprise entre 0 et 100

la quantité d'énergie max par gramme étant de 37.7kJ, la quantité d'énergie max pour 100g et 3770Kj

In [ ]:
for name in data_pca.loc[:,'energy_100g':'energy-from-fat_100g']:
    data_pca.drop( data_pca.loc[ (data_pca[name]<0) | (data_pca[name]>3830)].index, inplace=True)
display(data_pca.loc[:,'energy_100g':'energy-from-fat_100g'].describe())

les déviations standard !?

In [ ]:
# for name in data_pca.loc[:,'fat_100g':]:
    # print(name)
    # display(data.loc[ data_pca[data_pca[name] >= 99].index])
# data[data_pca['calcium_100g'] >= 99]
data.serving_size.value_counts()

In [ ]:
display(data_pca.loc[:,:'ingredients_that_may_be_from_palm_oil_n'].describe())
display(data_pca.loc[:,'energy_100g':].describe())


In [ ]:
data_pca = data.select_dtypes(np.number)

In [ ]:
# Trashbin - TODO : delete

# suppression des lignes avec code barre dupliqué
# data = data[~data.index.duplicated(keep='first')]
# data = data[~data.index.isna()]
# data[data.index.isna()]

# selection des valeurs numériques pour pca
# data_pca = data.select_dtypes(include = np.number).copy()

# drop des colones ne contenant 
# data_pca.drop(data_pca.count().where(lambda x: x<=1).dropna().index, axis='columns', inplace=True) #vérifier dropna ?
# display (data_pca.count().where(lambda x: x<=1).dropna().index) #vérifier dropna ?